In [ ]:
import networkx as nx
import numpy as np
import random
from scipy.optimize import linear_sum_assignment
import seaborn as sns
import matplotlib.pyplot as plt

prefer_list = None
rank_list = None
allocation = None
available_set = None
rank_sum = 0

sns.set_theme()
plt.rcParams['font.sans-serif'] = 'SimHei'


In [ ]:
def random_partition(n):
    idx = np.arange(n)
    random.shuffle(idx)
    return idx



"""
set with int value will be ordered no matter the original sequence.
use pop() to randomly pick since preferrences ensure that the index of agent makes no sense.
"""
def serial_dictator_partition(n):
    available_set = set(np.arange(n))
    not_chose_yet = available_set.copy()
    dictator = not_chose_yet.pop()
    idx = [dictator]
    while not_chose_yet:
        for j in prefer_list[dictator]:
            if j in available_set:
                available_set.remove(j)
                # the latest remain house won'be chosen since logic here, len(available_set) == 1
                if j not in not_chose_yet:
                    dictator = not_chose_yet.pop()
                else:
                    dictator = j
                    not_chose_yet.remove(dictator)
                idx.append(dictator) 
                break  
    return idx



def random_serial_dictator_partition(n):
    available_set = set(np.arange(n))
    not_chose_yet = available_set.copy()
    dictator = not_chose_yet.pop()
    idx1 = [dictator]
    idx2 = []
    while not_chose_yet:
        for j in prefer_list[dictator]:
            if j in available_set:
                idx2.append(j)
                available_set.remove(j)                
                dictator = not_chose_yet.pop()
                idx1.append(dictator) 
                break
    idx2.append(available_set.pop())
    not_chose_yet = set(np.arange(n))
    j = idx1[0]
    idx = [j]
    not_chose_yet.remove(j)

    while not_chose_yet:
        j = idx2[idx1.index(j)]
        if j in not_chose_yet:
            not_chose_yet.remove(j)    
        else:
            j = not_chose_yet.pop()
        idx.append(j)
    return idx


def assignment_problem(n):
    idx = []
    row_ind, col_ind = linear_sum_assignment(rank_list)
    not_chose_yet = set(np.arange(n))
    idx = [0]
    not_chose_yet.remove(0)
    j = 0
    while not_chose_yet:
        j = col_ind[j]
        if j in not_chose_yet:
            not_chose_yet.remove(j)    
        else:
            j = not_chose_yet.pop()
        idx.append(j)
    return idx

def TTC(available_set) -> None:
    G = nx.DiGraph()
    G.add_nodes_from(available_set)
    
    while available_set:
        # point to most prefer
        for i in available_set:
            # not point yet
            if G.out_degree(i) == 0:
                for j in prefer_list[i]:
                    if j in available_set:
                        G.add_edge(i, j)
                        break
        # remove cycles
        for cycle in nx.simple_cycles(G):
            for i in range(-len(cycle), 0):
                allocation[cycle[i]] = cycle[i+1]
                available_set.remove(cycle[i])
            G.remove_nodes_from(cycle)

In [ ]:
def TTC_partition(n):
    TTC(set(np.arange(n)))
    idx = []
    not_chose_yet = set(np.arange(n))
    idx = [0]
    not_chose_yet.remove(0)
    j = 0
    while not_chose_yet:
        j = allocation[j]
        if j in not_chose_yet:
            not_chose_yet.remove(j)    
        else:
            j = not_chose_yet.pop()
        idx.append(j)
    return idx

In [ ]:
epoches = 0
n_parms = [i*60 for i in range(2,8)]
k_parms = [2, 3, 4, 5]
for k_parm in k_parms:
    # for 4 kind of partition
    # rank_sum = [[],[],[],[]]
    rank_sum = []
    for n_parm in n_parms:
        prefer_list = [[i for i in range(n_parm)] for _ in range(n_parm)]
        rank_list = [[0]*n_parm for _ in range(n_parm)]
        allocation = [0]*n_parm
        # for i in rank_sum:
        #     i.append(0)
        rank_sum.append([0, 0, 0, 0])
        for _ in range(epoches):
            for i, j in zip(prefer_list, rank_list):
                random.shuffle(i)
                for h_i in range(n_parm):
                    j[i[h_i]] = h_i
            
            idx = random_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][0] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][1] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))
            
            idx = random_serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][2] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = assignment_problem(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][3] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))
        
    print(rank_sum)
    rank_sum = [[l/i**2 for l in j] for i, j in zip(n_parms, rank_sum)]
            
    plt.plot(n_parms, rank_sum, label=["随机划分","顺序独裁划分","随机独裁划分","分配划分"])
    plt.xlabel("n")
    plt.ylabel(chr(915))
    plt.legend(loc="best")
    # plt.savefig(str(k_parm)+"_mech")
    plt.show()
    

In [ ]:
epoches = 0
n_parms = [i*30 for i in range(2, 7)]
k_parms = [15, 30]
for k_parm in k_parms:
    # for 4 kind of partition
    # rank_sum = [[],[],[],[]]
    rank_sum = []
    for n_parm in n_parms:
        prefer_list = [[i for i in range(n_parm)] for _ in range(n_parm)]
        rank_list = [[0]*n_parm for _ in range(n_parm)]
        allocation = [0]*n_parm
        # for i in rank_sum:
        #     i.append(0)
        rank_sum.append([0, 0, 0, 0])
        for _ in range(epoches):
            for i, j in zip(prefer_list, rank_list):
                random.shuffle(i)
                for h_i in range(n_parm):
                    j[i[h_i]] = h_i
            
            idx = random_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][0] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][1] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))
            
            idx = random_serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][2] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = assignment_problem(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][3] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))
        
        # for i in rank_sum:
        #     i[-1] /= epoches
        # print(rank_sum)
        # for i in rank_sum:
        #     i[-1] /= n_parm**2
    print(rank_sum)
    rank_sum = [[l/i**2 for l in j] for i, j in zip(n_parms, rank_sum)]
            
    # rank_sum = list(map(list, zip(*rank_sum)))
    plt.plot(n_parms, rank_sum, label=["随机划分","顺序独裁划分","随机独裁划分","分配划分"])
    plt.xlabel("n")
    plt.ylabel(chr(915))
    plt.legend(loc="best")
    # plt.savefig(str(k_parm)+"_mech")
    plt.show()

In [ ]:
epoches = 100
n_parms = [i*30 for i in range(2, 7)]
k_parms = [3, 5];k_parms.pop()
for k_parm in k_parms:
    # for 4 kind of partition
    # rank_sum = [[],[],[],[]]
    rank_sum = []
    for n_parm in n_parms:
        prefer_list = [[i for i in range(n_parm)] for _ in range(n_parm)]
        rank_list = [[0]*n_parm for _ in range(n_parm)]
        allocation = [0]*n_parm
        # for i in rank_sum:
        #     i.append(0)
        rank_sum.append([0, 0, 0, 0])
        for _ in range(epoches):
            for i, j in zip(prefer_list, rank_list):
                random.shuffle(i)
                for h_i in range(n_parm):
                    j[i[h_i]] = h_i
            
            idx = random_partition(n_parm)
            res = []
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                res.append(sum(rank_list[j][j]-rank_list[j][allocation[j]] for j in available_set))
            rank_sum[-1][0] += np.var(res)

            # idx = serial_dictator_partition(n_parm)
            idx = TTC_partition(n_parm)
            res = []
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                res.append(sum(rank_list[j][j]-rank_list[j][allocation[j]] for j in available_set))
            rank_sum[-1][1] += np.var(res)
            
            idx = random_serial_dictator_partition(n_parm)
            res = []
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                res.append(sum(rank_list[j][j]-rank_list[j][allocation[j]] for j in available_set))
            rank_sum[-1][2] += np.var(res)

            idx = assignment_problem(n_parm)
            res = []
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                res.append(sum(rank_list[j][j]-rank_list[j][allocation[j]] for j in available_set))
            rank_sum[-1][3] += np.var(res)
        
    print(rank_sum)
    rank_sum = [[l/i**3 for l in j] for i, j in zip(n_parms, rank_sum)]
            
    plt.plot(n_parms, rank_sum, label=["随机划分","顺序独裁划分","随机独裁划分","分配划分"])
    plt.xlabel("n")
    plt.ylabel("var")
    plt.legend(loc="best")
    # plt.savefig(str(k_parm)+"_fair")
    plt.show()

In [ ]:
def worst_case_preference(n, k):
    s = int(n/k)
    idx = [i for i in range(n)]
    random.shuffle(idx)
    idx = [idx[i*s:(i+1)*s] for i in range(k)]
    # print(idx)
    temp = [[] for _ in range(n)]
    # i=-1 assign last, i=0 assign first
    for i in range(-k+1,1):
        # latter top
        temp[i*s].append(idx[i+1][0])
        temp[i*s].append(idx[i][-1])
    prev = []
    # one each group
    for i in range(k):    
        for j in range(1,s):
            temp[i*s+j].append(idx[i][j-1])
        # X/i*
        for l in range(k):
            if l != i:
                temp[i*s+1].append(idx[l][0])
        # push X
        for j in range(2,s):
            for l in range(k):
                temp[i*s+j].append(idx[l][0])
        for j in range(1,s):
            temp[i*s+j] += prev
            temp[i*s+j].append(idx[i][j])
        prev += idx[i][1:]
      
    for i in range(n):
        prefer_list[idx[i//s][i%s]] = temp[i] + [j for j in prefer_list[idx[i//s][i%s]] if j not in temp[i]]
    for i, j in zip(prefer_list, rank_list):
        for h_i in range(n):
            j[i[h_i]] = h_i

In [ ]:
epoches = 20
n_parms = [i*12 for i in range(1,6)]
k_parms = [2,3]
for k_parm in k_parms:
    # for 4 kind of partition
    rank_sum = []
    for n_parm in n_parms:
        prefer_list = [[i for i in range(n_parm)] for _ in range(n_parm)]
        rank_list = [[0]*n_parm for _ in range(n_parm)]
        allocation = [0]*n_parm
        # for i in rank_sum:
        rank_sum.append([0, 0, 0, 0, 0])
        for _ in range(epoches):
            worst_case_preference(n_parm, k_parm)
            
            idx = random_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][0] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][1] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = random_serial_dictator_partition(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][2] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))

            idx = assignment_problem(n_parm)
            for i in range(k_parm):
                available_set = set(idx[int(i*n_parm/k_parm):int((i+1)*n_parm/k_parm)])
                TTC(available_set)
            rank_sum[-1][3] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))


            available_set = set(np.arange(n_parm))
            TTC(available_set)
            rank_sum[-1][4] += sum(rank_list[i][i]-rank_list[i][allocation[i]] for i in range(n_parm))
            
    # for i in rank_sum:
    #     if i[1] == i[-1]:
    #         print(prefer_list)
    #         print(rank_list)
    #         break
    print(rank_sum)
    rank_sum = [[l/i**2 for l in j] for i, j in zip(n_parms, rank_sum)]
            
    plt.plot(n_parms, rank_sum, label=["随机划分","顺序独裁划分","随机独裁划分","分配划分","未划分"])
    plt.xlabel("n")
    plt.ylabel(chr(915))
    plt.legend(loc="best")
    # plt.savefig(str(k_parm)+"_worst")
    plt.show()